In [2]:
import pandas as pd
import sys
sys.path.append('../py')
import db
import weighted
import inspect
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
%matplotlib inline
from mpl_toolkits.basemap import Basemap
import seaborn as sns
import scipy.stats as stats
import numpy as np
import math

In [3]:
cities = "('北京','上海','深圳','杭州','广州','南京','成都','东莞','西安','武汉','天津','长沙','宁波','福州','大连','重庆','青岛','济南','合肥','长春','昆明','郑州','沈阳','哈尔滨')"

In [4]:
conn=db.get_conn()
data_4=pd.read_sql(sql=f"select sum(monthly_salary * headcount)/sum(headcount) as April,max(city) as city from _201904 where monthly_salary>0 and monthly_salary<80000 and career='一般程序员' and pl_java=1 group by city,career", con=conn)
data_5=pd.read_sql(sql=f"select sum(monthly_salary * headcount)/sum(headcount) as May,max(city) as city from _201905 where monthly_salary>0 and monthly_salary<80000 and city in {cities} and career='一般程序员' and pl_java=1 group by city", con=conn)
data_6=pd.read_sql(sql=f"select sum(monthly_salary * headcount)/sum(headcount) as June,max(city) as city from _201906 where monthly_salary>0 and monthly_salary<80000 and city in {cities} and career='一般程序员' and pl_java=1 group by city", con=conn)
data_7=pd.read_sql(sql=f"select sum(monthly_salary * headcount)/sum(headcount) as July,max(city) as city from _201907 where monthly_salary>0 and monthly_salary<80000 and city in {cities} and career='一般程序员' and pl_java=1 group by city", con=conn)
data_8=pd.read_sql(sql=f"select sum(monthly_salary * headcount)/sum(headcount) as August,max(city) as city from _201908 where monthly_salary>0 and monthly_salary<80000 and city in {cities} and career='一般程序员' and pl_java=1 group by city", con=conn)
conn.close()

In [5]:
translate_dict={}
translate_dict['beijing']='北京'
translate_dict['shanghai']='上海'
translate_dict['shenzhen']='深圳'
translate_dict['hangzhou']='杭州'
translate_dict['guangzhou']='广州'
translate_dict['nanjing']='南京'
translate_dict['chengdu']='成都'
translate_dict['dongguan']='东莞'
translate_dict['xian']='西安'
translate_dict['wuhan']='武汉'
translate_dict['tianjin']='天津'
translate_dict['changsha']='长沙'
translate_dict['ningbo']='宁波'
translate_dict['fuzhou']='福州'
translate_dict['dalian']='大连'
translate_dict['chongqing']='重庆'
translate_dict['qingdao']='青岛'
translate_dict['jinan']='济南'
translate_dict['hefei']='合肥'
translate_dict['changchun']='长春'
translate_dict['kunming']='昆明'
translate_dict['zhengzhou']='郑州'
translate_dict['shenyang']='沈阳'
translate_dict['harbin']='哈尔滨'

In [6]:
data_4.city=data_4.city.map(translate_dict)

In [7]:
data_4=data_4.set_index('city')
data_5=data_5.set_index('city')
data_6=data_6.set_index('city')
data_7=data_7.set_index('city')
data_8=data_8.set_index('city')

In [8]:
new_data=data_4.join(data_5).join(data_6).join(data_7).join(data_8)

In [9]:
new_data['change']=new_data.August-new_data.April

In [10]:
new_data.sort_values(by='August', ascending=False)

,April,May,June,July,August,change
city,,,,,,
上海,15272,15419,15477,15697,15796,524
深圳,15237,15198,15488,15551,15773,536
北京,14662,14504,14947,14983,15076,414
杭州,14315,14315,14355,14661,14735,420
广州,12435,12530,12562,12697,12956,521
南京,11714,11750,12347,12484,12607,893
东莞,12485,11196,11634,11843,11744,-741
成都,11861,11293,11170,11239,11302,-559
武汉,10657,11342,10682,10910,10950,293


In [11]:
new_data.sort_values(by='change', ascending=False)

,April,May,June,July,August,change
city,,,,,,
南京,11714,11750,12347,12484,12607,893
合肥,8844,7919,8983,9328,9501,657
深圳,15237,15198,15488,15551,15773,536
上海,15272,15419,15477,15697,15796,524
广州,12435,12530,12562,12697,12956,521
杭州,14315,14315,14355,14661,14735,420
北京,14662,14504,14947,14983,15076,414
长沙,9970,9942,10078,10141,10377,407
武汉,10657,11342,10682,10910,10950,293
